# Capstone Project - Week 3 Assignment
## Part 1 - Webscraping

Import the required libraries

In [48]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

Scraping the data from wikipedia into a data frame and inspecting the borough names

In [201]:
url = ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

html = requests.get(url).text
soup = BeautifulSoup(html, 'html5lib')
table = soup.find('table')
                      
t_cont = []
for row in table.find_all('tr'):
    for col in row.find_all('td'):
        cell = {}
        if col.span.text=='Not assigned':
            pass
        else:
            cell['PostalCode'] = col.p.text[:3]
            cell['Borough'] = (col.span.text).split('(')[0]
            cell['Neighborhood'] = (((((col.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
            t_cont.append(cell)

toronto_df = pd.DataFrame(t_cont)
fix_order = ['PostalCode', 'Borough', 'Neighborhood']
toronto_df = toronto_df[fix_order]
toronto_df['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'East YorkEast Toronto', 'Central Toronto',
       'MississaugaCanada Post Gateway Processing Centre',
       'Downtown TorontoStn A PO Boxes25 The Esplanade',
       'EtobicokeNorthwest',
       'East TorontoBusiness reply mail Processing Centre969 Eastern'],
      dtype=object)

Cleaning the 'Borough' names and checking for 'Neighborhood's with no values

In [202]:
toronto_df['Borough'].replace({'East YorkEast Toronto':'East York/East Toronto',
                              'MississaugaCanada Post Gateway Processing Centre':'Mississauga',
                              'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                              'EtobicokeNorthwest':'Etobicoke Northwest',
                              'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business'}, inplace=True)

print('{} neighborhoods have missing values'.format(toronto_df['Neighborhood'].isnull().sum()))

0 neighborhoods have missing values


Printing the number of rows in the data frame

In [203]:
toronto_df.shape[0]

103

## Part 2 - Geospacial Data

In this part of the notebook, the latitude and longitude of each postal code is downloaded using the Geocoder python package, we start by installing and importing the package

In [87]:
!pip install geocoder
import geocoder

Then for each postal code in the dataframe, we need to use a while loop, initializing the coordinates as None, since the package can be ify sometimes

In [92]:
lat = []
long = []
for pc in toronto_df['PostalCode']:
    print('Beginning to retrieve {} coordinates '.format(pc))
    coords = None
    while coords is None:
        geo = geocoder.google('{}, Toronto, Ontario'.format(pc))
        coords = geo.latlng
    lat.append(coords[0])
    long.append(coords[1])
    print('The coordinates for {} have been retrieved'.format(pc))
toronto_df['Latitude'] = lat
toronto_df['Longitude'] = long

Beginning to retrieve M3A coordinates 


KeyboardInterrupt: 

Since the package did not want to work properly, I downloaded the geospatial data from the provided csv file, as follows

In [204]:
coords_df = pd.read_csv('Geospatial_Coordinates.csv')
coords_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [205]:
toronto_df = toronto_df.join(coords_df.set_index('Postal Code'), on='PostalCode')
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


## Part 3 - Exploring and Clustering Toronto Neighborhoods

### Exploratory Data Analysis

In order to simplify the analysis I will use a subset of the data which will only include boroughs that contain the word 'Toronto'

In [206]:
df = toronto_df[toronto_df['Borough'].str.contains('Toronto', na=False)].reset_index()
print(df.head())
print(df.tail())

   index PostalCode           Borough               Neighborhood   Latitude  \
0      2        M5A  Downtown Toronto  Regent Park, Harbourfront  43.654260   
1      9        M5B  Downtown Toronto   Garden District, Ryerson  43.657162   
2     15        M5C  Downtown Toronto             St. James Town  43.651494   
3     19        M4E      East Toronto                The Beaches  43.676357   
4     20        M5E  Downtown Toronto                Berczy Park  43.644771   

   Longitude  
0 -79.360636  
1 -79.378937  
2 -79.375418  
3 -79.293031  
4 -79.373306  
    index PostalCode                 Borough  \
34     92        M5W  Downtown Toronto Stn A   
35     96        M4X        Downtown Toronto   
36     97        M5X        Downtown Toronto   
37     99        M4Y        Downtown Toronto   
38    100        M7Y   East Toronto Business   

                              Neighborhood   Latitude  Longitude  
34                          Enclave of M5E  43.646435 -79.374846  
35          

We need my foursquare API client ID and secret as well as to set the limit on the number of results retrieved for any call made to the api and the version of the api to be used

In [207]:
CLIENT_ID = 'QMDWTNEPHUORERIL1RGVZD0WVOUMOPXVCT4J0RJYGH4LO233'
CLIENT_SECRET = 'UBFU2XT2ED334MXQTMF34SRXZ3JRW1SQZJH02CF0TI4TABSC'
VERSION = '20180604'
LIMIT = 100

Fisrt we need to begin by defining a function that will retrieve the nearby venues in each neighborhood which includes defining the foursquare api url

In [208]:
def NearbyVenues(names, lats, longs, rad=500):
    
    venues_list = []
    
    for name, lat, long in zip(names, lats, longs):
        print('{} venue data retrieved'.format(name))
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                long, 
                rad, 
                LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(name, lat, long,
                            v['venue']['name'],
                            v['venue']['location']['lat'],
                            v['venue']['location']['lng'],
                            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood','N_lat', 'N_long', 'Venue', 'V_lat', 'V_long', 'V_cat']
    
    return nearby_venues

Call the function defined above to create a data frame of venues in thier respective neighborhoods

In [209]:
toronto_venues = NearbyVenues(names=df['Neighborhood'], lats=df['Latitude'], longs=df['Longitude'])

Regent Park, Harbourfront venue data retrieved
Garden District, Ryerson venue data retrieved
St. James Town venue data retrieved
The Beaches venue data retrieved
Berczy Park venue data retrieved
Central Bay Street venue data retrieved
Christie venue data retrieved
Richmond, Adelaide, King venue data retrieved
Dufferin, Dovercourt Village venue data retrieved
The Danforth  East venue data retrieved
Harbourfront East, Union Station, Toronto Islands venue data retrieved
Little Portugal, Trinity venue data retrieved
The Danforth West, Riverdale venue data retrieved
Toronto Dominion Centre, Design Exchange venue data retrieved
Brockton, Parkdale Village, Exhibition Place venue data retrieved
India Bazaar, The Beaches West venue data retrieved
Commerce Court, Victoria Hotel venue data retrieved
Studio District venue data retrieved
Lawrence Park venue data retrieved
Roselawn venue data retrieved
Davisville North venue data retrieved
Forest Hill North & West venue data retrieved
High Park, The

Inspect the size and'or shape of the resulting dataframe as well as the first few rows

In [210]:
print(toronto_venues.shape)
toronto_venues.head()

(1575, 7)


,Neighborhood,N_lat,N_long,Venue,V_lat,V_long,V_cat
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Inspect how many venues were returned for each neighborhood

In [211]:
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Berczy Park                                                                                                    60
Brockton, Parkdale Village, Exhibition Place                                                                   23
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport     16
Central Bay Street                                                                                             61
Christie                                                                                                       16
Church and Wellesley                                                                                           76
Commerce Court, Victoria Hotel                                                                                100
Davisville                                                                                                     34
Davisville North                                                           

How many unique categories can be found in the dataframe?

In [212]:
len(toronto_venues['V_cat'].unique())

234

Create the 1-hot encoding for the neighborhoods and their venues and fix the column order

In [213]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

tor_onehot = pd.get_dummies(toronto_venues[['V_cat']], prefix='', prefix_sep='')
tor_onehot.drop('Neighborhood', axis=1, inplace=True)
tor_onehot['Neighborhood'] = toronto_venues['Neighborhood']


fixed_order = ['Neighborhood']+list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_order]

tor_onehot.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,

Inspect the new dataframe size to check if it matches the expected 235 columns 1575 rows

In [214]:
tor_onehot.shape

(1575, 234)

We were expecting 235, but after thorough investigation, I found that one of the categories that foursquare tagged a venue as was 'Neighborhood', therefore I dropped that column from the 1-hot encoded dataframe



Now, I get the mean frequency of each category for every neighborhood

In [215]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.016667,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.016667,0.050000,0.000000,0.000000,0.00,0.016667,0.016667,0.000000,0.033333,0.000000,0.000000,0.016667,0.00,0.0000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.016667,0.00,0.0000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.016667,0.066667,0.100000,0.00000,0.00000,0.00,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.000000,0.0

Confirming the new size as 39

In [216]:
tor_grouped.shape[0]

39

Exploring the top 5 venue categories in each neighborhood

In [217]:
for hood in tor_grouped['Neighborhood']:
    print('***'+hood+'***')
    ndf = tor_grouped[tor_grouped['Neighborhood']==hood].T.reset_index()
    ndf.columns = ['venue', 'frequency']
    ndf= ndf.iloc[1:]
    ndf['frequency'] = ndf['frequency'].astype(float)
    ndf = ndf.round({'frequency':3})
    print(ndf.sort_values('frequency', ascending=False).reset_index(drop=True).head())
    print('\n')

***Berczy Park***
          venue  frequency
0   Coffee Shop      0.100
1  Cocktail Bar      0.067
2        Bakery      0.050
3      Pharmacy      0.033
4      Beer Bar      0.033


***Brockton, Parkdale Village, Exhibition Place***
            venue  frequency
0            Café      0.130
1  Breakfast Spot      0.087
2     Coffee Shop      0.087
3       Nightclub      0.043
4         Stadium      0.043


***CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport***
              venue  frequency
0    Airport Lounge      0.125
1   Airport Service      0.125
2  Airport Terminal      0.125
3             Plane      0.062
4           Airport      0.062


***Central Bay Street***
                venue  frequency
0         Coffee Shop      0.180
1      Sandwich Place      0.066
2                Café      0.049
3  Italian Restaurant      0.049
4     Bubble Tea Shop      0.033


***Christie***
                venue  frequency
0       Grocery S

Create a function that returns the 10 most frequenct venue categries for each neighborhood

In [218]:
def most_common(row, top=10):
    row_cat = row.iloc[1:]
    sorted = row_cat.sort_values(ascending=False)
    return sorted.index.values[0:top]

Call the above function for each neighborhood in 'tor_grouped' and store the results in a new dataframe

In [219]:
columns = ['Neighborhood','1','2','3','4','5','6','7','8','9','10']

tor_venues_sorted = pd.DataFrame(columns=columns)
tor_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for i in np.arange(tor_grouped.shape[0]):
    tor_venues_sorted.iloc[i,1:] = most_common(tor_grouped.iloc[i,:])

tor_venues_sorted

,Neighborhood,1,2,3,4,5,6,7,8,9,10
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Beer Bar,Farmers Market,Pharmacy,Seafood Restaurant,Cheese Shop,Clothing Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Convenience Store,Intersection,Bar,Restaurant,Bakery,Climbing Gym,Burrito Place
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Coffee Shop,Plane,Harbor / Marina,Sculpture Garden,Airport Gate
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Modern European Restaurant,Juice Bar,Korean Restaurant
4,Christie,Grocery Store,Café,Park,Baby Store,Coffee Shop,Italian Restaurant,Athletics & Sports,Nightclub,Restaurant,Candy Store
5,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Pub,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio
6,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Hotel,Café,Gym,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,Deli / Bodega,American Restaurant
7,Davisville,Sandwich Place,Dessert Shop,Sushi Restaurant,Pizza Place,Gym,Italian Restaurant,Café,Coffee Shop,Brewery,Restaurant
8,Davisville North,Hotel,Gym / Fitness Center,Breakfast Spot,Gym,Food & Drink Shop,Dog Run,Department Store,Dance Studio,Park,Sandwich Place
9,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Café,Supermarket,Bank,Bar,Middle Eastern Restaurant,Music Venue,Liquor Store,Brewery


### Clustering with KMeans
Importing the library and clustering the neighborhoods into 5 clusters using KMeans

In [220]:
from sklearn.cluster import KMeans

tor_cluster = tor_grouped.drop('Neighborhood', axis=1)

km = KMeans(n_clusters=5, random_state=4)
km.fit(tor_cluster)

km.labels_[0:15]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0])

Now create a new dataframe containing the corresponding cluster labels for each neighborhood

In [223]:
tor_venues_sorted.insert(0,'Cluster Labels', km.labels_)

toronto = tor_venues_sorted
toronto = toronto.join(toronto_df.set_index('Neighborhood'), on='Neighborhood')
fixed_order = ['Borough','Neighborhood', 'Latitude', 'Longitude', 'Cluster Labels','1','2','3','4','5','6','7','8','9','10']
toronto = toronto[fixed_order]

toronto.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1,2,3,4,5,6,7,8,9,10
0,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Beer Bar,Farmers Market,Pharmacy,Seafood Restaurant,Cheese Shop,Clothing Store
1,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,0,Café,Coffee Shop,Breakfast Spot,Convenience Store,Intersection,Bar,Restaurant,Bakery,Climbing Gym,Burrito Place
2,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Coffee Shop,Plane,Harbor / Marina,Sculpture Garden,Airport Gate
3,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Modern European Restaurant,Juice Bar,Korean Restaurant
4,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Baby Store,Coffee Shop,Italian Restaurant,Athletics & Sports,Nightclub,Restaurant,Candy Store


Now I can import the folium and required matplotlib libraries and visualize the resulting clusters (from a simple google search I found Toronto's coordinates as 43.6532N and 79.3832W)

In [234]:
import matplotlib.cm as cm
import matplotlib.colors as c
import folium

latitude = 43.6532
longitude = -79.3832
tor_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [c.rgb2hex(i) for i in colors_array]

for lat, lon, hood, cluster in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood'], toronto['Cluster Labels']):
    label = folium.Popup(str(hood) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(tor_clusters)
       
tor_clusters

This concludes the week 3 assignment, thank you!